# GLM for DS008

This is based on the famous stop-signal dataset of Adam Aron and was fetched from openfmri.

See [ds008](https://openfmri.org/dataset/ds000008)

In [38]:
!tree /home/ubuntu/data/openfmri/ds008/ -d 2

/home/ubuntu/data/openfmri/ds008/
├── derivatives
│   └── fmriprep
│       ├── sub-01
│       │   ├── anat
│       │   └── func
│       ├── sub-02
│       │   ├── anat
│       │   └── func
│       ├── sub-03
│       │   ├── anat
│       │   └── func
│       ├── sub-04
│       │   ├── anat
│       │   └── func
│       ├── sub-05
│       │   ├── anat
│       │   └── func
│       ├── sub-06
│       │   ├── anat
│       │   └── func
│       ├── sub-07
│       │   ├── anat
│       │   └── func
│       ├── sub-09
│       │   ├── anat
│       │   └── func
│       ├── sub-10
│       │   ├── anat
│       │   └── func
│       ├── sub-11
│       │   ├── anat
│       │   └── func
│       ├── sub-12
│       │   ├── anat
│       │   └── func
│       ├── sub-13
│       │   ├── anat
│       │   └── func
│       ├── sub-14
│       │   ├── anat
│       │   └── func
│       └── sub-15
│           ├── anat
│           └── func
├── freesurfer
├── sub-01
│   ├── anat
│   └── func
├── sub-02
│   ├── anat
│  